# Simple label attack

### Time: 
$t=1,\ldots h$

### Teacher: 
$y^{\textrm{t}}=1$

### Attacker's target: 
$y^{\star}=-1$

### Control: 
$\mathbf{a}=(a_1, \ldots a_h), \, a_t\in[0, 1]$

### Attack: 
$y_t^{\dagger}=y^{\textrm{t}}(1-a_t) + y^{\star}a_t$

### Dynamics: 
$y_{t+1}^{\mathrm{s}} = y_t^{\dagger} \beta + y_t^{\mathrm{s}}(1-\beta)$

### Objective:
$min_{\mathbf{a}}\sum_t \gamma^t( g_{\mathrm{nef}}(y_t^{\mathrm{s}}) + g_{\mathrm{per}}(a_t))$
<br>
$g_{\mathrm{nef}}(y_t^{\mathrm{s}}) = \frac{1}{2}(y_t^{\mathrm{s}} - y^{\star})^2 \in [0, 2]$
<br>
$g_{\mathrm{per}}(a_t) = a_t^2 \in [0, 1]$

### Initial conditions:
$y_0^{\mathrm{s}} = 0$
<br>
$\mathbf{a}_0 = (0, \ldots 0)$

### Parameters:
$\gamma, \beta, y^{\star}, y^{\mathrm{t}}, h$

NB1: We do not care for $a_0$ and $y_0^{\dagger}$, though for consistency of indices we shall use vectors with $h+1$ entries (same as $\mathbf{y}^{\mathrm{s}}$).

NB2: The sequence of events is: $y_{0}^{\mathrm{s}}, a_1, y_{1}^{\mathrm{s}}, \ldots$
<br>
At $t=0$ whe have $a_0$ and $y_0^{\dagger}$ that play no role, while at $t=h$ we perform the last update $y_{h}^{\mathrm{s}} = y_{h}^{\dagger} \beta + y_{h-1}^{\mathrm{s}}(1-\beta)$.

### Long-time behaviour (trivial)
We can write $y_t^{\mathrm{s}}$ as:
<br>
$y_t^{\mathrm{s}} = y_0^{\mathrm{s}}(1-\beta)^t + \sum_{t'=1}^t y^{\dagger}(a_{t'})\beta(1-\beta)^{t-t'}$
<br>
If we set $a_{t'}=0\, \forall \, t'$, then $y^{\dagger}(a_{t'}=0)=y^{\mathrm{t}}$ and
<br>
$y_{\infty}^{\mathrm{s}}=\lim_{h\to\infty} y_0^{\mathrm{s}}(1-\beta)^h + \sum_{t'=1}^h y^{\mathrm{t}}\beta(1-\beta)^{h-t'}=\sum_{t'=0}^{\infty} y^{\mathrm{t}}\beta(1-\beta)^{t'}=y^{\mathrm{t}}$
<br>
Similarly if $a_{t'}=1\, \forall \, t'$, then $y^{\dagger}(a_{t'}=1)=y^{\star}$ and $y_{\infty}^{\mathrm{s}}=y^{\star}$

### Gradient of objective function
$\partial_{a_t} \sum_{t'=1}^h \gamma^{t'} (g_{\mathrm{per}}(a_{t'}) + g_{\mathrm{nef}}(y_{t'}^{\mathrm{s}})) = 2 \gamma^{t} a_t + \partial_{a_t} \sum_{t'=1}^h \gamma^{t'} g_{\mathrm{nef}}(y_{t'}^{\mathrm{s}})$
<br>
$\partial_{a_t} \sum_{t'=1}^h \gamma^{t'} g_{\mathrm{nef}}(y_{t'}^{\mathrm{s}}) = \sum_{t'=t}^h \gamma^{t'} \partial_{a_t} g_{\mathrm{nef}}(y_{t'}^{\mathrm{s}}) = \sum_{t'=t}^h \gamma^{t'} \partial_{a_t} \frac{1}{2}(y_t^{\mathrm{s}} - y^{\star})^2 = \sum_{t'=t}^h \gamma^{t'} (y_{t'}^{\mathrm{s}} - y^{\star})\partial_{a_t}y_{t'}^{\mathrm{s}} = \sum_{t'=t}^h \gamma^{t'} (y_{t'}^{\mathrm{s}} - y^{\star})\beta(1-\beta)^{t'-t}(y^{\star}-y^{\mathrm{t}})$
<br>
as $\partial_{a_t} y_{t'}^{\mathrm{s}} = \partial_{a_t} \big(y_0^{\mathrm{s}}(1-\beta)^{t'} + \sum_{t''=1}^{t'} y^{\dagger}(a_{t''})\beta(1-\beta)^{t'-t''} \big)= \beta(1-\beta)^{t'-t}\partial_{a_t}y^{\dagger}(a_{t}) = \beta(1-\beta)^{t'-t}\partial_{a_t}(y^{\mathrm{t}}(1-a_{t})+y^{\star}a_t) = \beta(1-\beta)^{t'-t}(y^{\star} - y^{\mathrm{t}})$
<br>
Finally
<br>
$\partial_{a_t} \sum_{t'=1}^h \gamma^{t'} (g_{\mathrm{per}}(a_{t'}) + g_{\mathrm{nef}}(y_{t'}^{\mathrm{s}})) = 2 \gamma^{t} a_t + \sum_{t'=t}^h \gamma^{t'} (y_{t'}^{\mathrm{s}} - y^{\star})\beta(1-\beta)^{t'-t}(y^{\star}-y^{\mathrm{t}})$

## Import statement

In [48]:
import ipopt
import scipy.sparse as sps
import numpy as np

## Implementation

In [49]:
# Global variables
gamma = 0.9
beta = 0.5
y_star = -1
y_teach = 1
y_stud_0 = 0
h = 10

class LabelAttack(object):
    def __init__(self):
        pass

    def objective(self, a):
        #
        # The callback for calculating the objective
        #
        
        # time vector
        time = np.arange(0, h+1) # t = 0, 1, ...h
        
        # Perturbation cost
        g_per = a[1:]**2
        g_per = np.sum(gper * gamma**time[1:])
        
        # Nefarious cost
        y_dagger = y_teach * (1-a) + y_star * a
        y_stud = np.zeros(h+1)
        y_stud[0] = y_stud_0
        for t in range(h):
            y_stud[t+1] = y_dagger[t+1] * beta + y_stud[t] * (1-beta)
        g_nef = np.sum(((y_stud[1:]-y_star)**2) * gamma**time[1:])/2.
        
        return g_per + g_nef

    def gradient(self, a):
        #
        # The callback for calculating the gradient
        #
        
        # time vector
        time = np.arange(0, h+1) # t = 0, 1, ...h
        
        # student vector
        y_dagger = y_teach * (1-a) + y_star * a
        y_stud = np.zeros(h+1)
        y_stud[0] = y_stud_0
        for t in range(h):
            y_stud[t+1] = y_dagger[t+1] * beta + y_stud[t] * (1-beta)
        
        # gradient vector
        g_grad = 2 * (gamma**time) * a
        #
        prefactor = beta * (y_star-y_teach)
        pref_disc = prefactor * gamma**time
        for t in range(h):
            time_f = time[t+1:h+1]
            time_b = time[1:h+1-t]
            g_nef_grad_t = np.sum(pref_disc[time_f]*((1-beta)**time_b)*(y_stud[time_f]-y_star))
            
            g_grad[t+1] = g_grad[t+1] + g_nef_grad_t
        
        return g_grad[1:]

    def constraints(self, a):
        #
        # The callback for calculating the constraints
        #
        
        # student vector
        y_dagger = y_teach * (1-a) + y_star * a
        y_stud = np.zeros(h+1)
        y_stud[0] = y_stud_0
        for t in range(h):
            y_stud[t+1] = y_dagger[t+1] * beta + y_stud[t] * (1-beta)
            
        return y_stud[1:]

    def jacobian(self, a):
        #
        # The callback for calculating the Jacobian
        #
        return np.ones(h) * (y_star - y_teach) * beta
    
    '''
    def hessianstructure(self):
        #
        # The structure of the Hessian
        # Note:
        # The default hessian structure is of a lower triangular matria. Therefore
        # this function is redundant. I include it as an eaample for structure
        # callback.
        #
        global hs

        hs = sps.coo_matria(np.tril(np.ones((4, 4))))
        return (hs.col, hs.row)

    def hessian(self, a, lagrange, obj_factor):
        #
        # The callback for calculating the Hessian
        #
        H = obj_factor*np.array((
                (2*a[3], 0, 0, 0),
                (a[3],   0, 0, 0),
                (a[3],   0, 0, 0),
                (2*a[0]+a[1]+a[2], a[0], a[0], 0)))

        H += lagrange[0]*np.array((
                (0, 0, 0, 0),
                (a[2]*a[3], 0, 0, 0),
                (a[1]*a[3], a[0]*a[3], 0, 0),
                (a[1]*a[2], a[0]*a[2], a[0]*a[1], 0)))

        H += lagrange[1]*2*np.eye(4)

        #
        # Note:
        #
        #
        return H[hs.row, hs.col]
    '''
    
    def intermediate(
            self,
            alg_mod,
            iter_count,
            obj_value,
            inf_pr,
            inf_du,
            mu,
            d_norm,
            regularization_size,
            alpha_du,
            alpha_pr,
            ls_trials
            ):

        #
        # Eaample for the use of the intermediate callback.
        #
        print("Objective value at iteration #%d is - %g" % (iter_count, obj_value))

In [50]:
# Initial conditions
a_in = np.ones(h+1)/2.
# student vector
y_dagger_in = y_teach * (1-a_in) + y_star * a_in
y_stud_in = np.zeros(h+1)
y_stud_in[0] = y_stud_0
for t in range(h):
    y_stud_in[t+1] = y_dagger_in[t+1] * beta + y_stud_in[t] * (1-beta)
            
# Lower bounds - variables
lb = np.zeros(h+1)

# Upper bounds - variables
ub = np.ones(h+1)

# Lower bounds - constraints
#cl = y_stud_in[:-1]
cl = np.zeros(h)

# Upper bounds - constraints
#cu = y_stud_in[:-1]
cu = np.ones(h)

nlp = ipopt.problem(
            n=len(a0),
            m=len(cl),
            problem_obj=LabelAttack(),
            lb=lb,
            ub=ub,
            cl=cl,
            cu=cu
            )

# Setting optimization parameters
nlp.addOption('mu_strategy', 'adaptive')
nlp.addOption('tol', 1e-7)

# Optimization run
a, info = nlp.solve(a_in)

# Solution
print(a)

IndexError: Out of bounds on buffer access (axis 0)

Exception ignored in: 'cyipopt.gradient_cb'
IndexError: Out of bounds on buffer access (axis 0)


IndexError: Out of bounds on buffer access (axis 0)

Exception ignored in: 'cyipopt.jacobian_cb'
IndexError: Out of bounds on buffer access (axis 0)


IndexError: Out of bounds on buffer access (axis 0)

[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]


Exception ignored in: 'cyipopt.jacobian_cb'
IndexError: Out of bounds on buffer access (axis 0)


In [19]:
h = 2
t = np.arange(0, h+1)
t, 2**t

(array([0, 1, 2]), array([1, 2, 4]))

In [22]:
h = 10
time = np.arange(0, h+1)
t = 3
time_f = time[t+1:h+1]
time_b = time[1:h+1-t]
time_f, time_b

(array([ 4,  5,  6,  7,  8,  9, 10]), array([1, 2, 3, 4, 5, 6, 7]))

In [23]:
np.ones(h)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])